
# CNN for Solving a 5-Bus Power System Problem

This Jupyter Notebook implements a simple Convolutional Neural Network (CNN) using TensorFlow/Keras to solve a 5-bus power system problem. The model is trained to predict power flow or bus voltage magnitudes based on input features such as load and generation data.

## Table of Contents:
1. [Load the 5-bus power system from pandapower](#step-1-load-the-5-bus-power-system-from-pandapower)
2. [Extract relevant input features from the power flow analysis](#step-2-extract-relevant-input-features-from-the-power-flow-analysis)
3. [Define a CNN model with a three-layer structure](#step-3-define-a-cnn-model-with-three-hidden-layers)
4. [Train the model using power system data](#step-4-train-the-model-using-power-system-data)
5. [Predict results for new input data](#step-5-predict-results-for-new-input-data)
6. [Compare predictions and Visualize model performance](#step-6-compare-predictions-for-different-training-epochs-and-filter-sizes)

In [ ]:
# Install required packages
!pip install pandapower tensorflow matplotlib numpy

In [ ]:
import pandapower as pp
import pandas as pd
import pandapower.networks as nw
import pandapower.plotting as pplot
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

## Step 1. Load the 5-bus power system from pandapower
CASE5  Power flow data for modified 5 bus, 5 gen case based on PJM 5-bus system

*F.Li and R.Bo, "Small Test Systems for Power System Economic Studies", DOI: [10.1109/PES.2010.5589973](https://doi.org/10.1109/PES.2010.5589973)*

![image-2.png](attachment:image-2.png)

In [ ]:
# Load the 5-bus power system network
net = nw.case5()
print(net)

In [ ]:
# Run power flow analysis
net.bus['max_vm_pu'] = 1.10
net.bus['min_vm_pu'] = 0.90
pp.runpp(net)

# Check available columns for loads and generators
print("Load columns:", net.load.columns)
print("Gen columns:", net.gen.columns)

## Step 2. Extract relevant input features from the power flow analysis.

In [ ]:
# Extract input features: voltage magnitude, load, and generation data
X = np.column_stack([
    net.res_bus.vm_pu.values,   # Bus voltage magnitudes
    net.load.p_mw.reindex(net.bus.index, fill_value=0).values,  # Load active power
    net.load.q_mvar.reindex(net.bus.index, fill_value=0).values if 'q_mvar' in net.load else np.zeros(len(net.bus)), # Load reactive power (if available)
    net.gen.p_mw.reindex(net.bus.index, fill_value=0).values,    # Generator active power
    net.gen.q_mvar.reindex(net.bus.index, fill_value=0).values if 'q_mvar' in net.gen else np.zeros(len(net.bus))   # Generator reactive power (if available)
])
X = X.reshape((X.shape[0], X.shape[1], 1))  # Reshape for CNN

# Output: bus voltage magnitudes and  power flows
Y_voltage = net.res_bus.vm_pu.values.reshape(-1, 1)
Y_power_flow = np.column_stack([
    net.res_line.p_from_mw.values,
    net.res_line.q_from_mvar.values
])

## Step 3. Define a CNN model with three hidden layers
### Bus voltage
- Model 1 (epochs=50, 24 filters),
   - one input layer (12 input nodes);
   -  three hidden layers (24, 24 and 12 nodes, respectively);
   -  one single-node output layer.
- Model 2 (epochs=50, 240 filters)

- Model 3 (epochs=200, 240 filters)
### Power Flow
- Model 4 (epochs=200, 24 filters)

<figure>
  <img src="https://dfzljdn9uc3pi.cloudfront.net/2023/cs-1395/1/fig-1-2x.jpg" width="450">
</figure>



In [ ]:
# Model 1 (epochs=50, 24 filters)
model_50 = keras.Sequential([
    layers.Conv1D(filters=24, kernel_size=2, activation='relu', input_shape=(X.shape[1], 1)),
    layers.Conv1D(filters=24, kernel_size=2, activation='relu'),
    layers.Conv1D(filters=12, kernel_size=2, activation='relu'),
    layers.Flatten(),
    layers.Dense(12, activation='relu'),
    layers.Dense(1, activation='linear')  # Output layer for regression
])
model_50.compile(optimizer='adam', loss='mse', metrics=['mae'])

# Model 2 (epochs=50, 240 filters)
model_50_v2 = keras.Sequential([
    layers.Conv1D(filters=240, kernel_size=2, activation='relu', input_shape=(X.shape[1], 1)),
    layers.Conv1D(filters=240, kernel_size=2, activation='relu'),
    layers.Conv1D(filters=120, kernel_size=2, activation='relu'),
    layers.Flatten(),
    layers.Dense(12, activation='relu'),
    layers.Dense(1, activation='linear')  # Output layer for regression
])
model_50_v2.compile(optimizer='adam', loss='mse', metrics=['mae'])
# Model 3 (epochs=200, 240 filters)
model_200 = keras.Sequential([
    layers.Conv1D(filters=240, kernel_size=2, activation='relu', input_shape=(X.shape[1], 1)),
    layers.Conv1D(filters=240, kernel_size=2, activation='relu'),
    layers.Conv1D(filters=120, kernel_size=2, activation='relu'),
    layers.Flatten(),
    layers.Dense(12, activation='relu'),
    layers.Dense(1, activation='linear')  # Output layer for regression
])
model_200.compile(optimizer='adam', loss='mse', metrics=['mae'])

# Model_Power_Flow: Model 4 (epochs=200, 24 filters)
model_power_flow = keras.Sequential([
    layers.Conv1D(filters=24, kernel_size=2, activation='relu', input_shape=(X.shape[1], 1)),
    layers.Conv1D(filters=24, kernel_size=2, activation='relu'),
    layers.Conv1D(filters=12, kernel_size=2, activation='relu'),
    layers.Flatten(),
    layers.Dense(12, activation='relu'),
    layers.Dense(2, activation='linear')  # Two outputs for p_mw and q_mvar
])
model_power_flow.compile(optimizer='adam', loss='mse', metrics=['mae'])

## Step 4. Train the model using power system data

In [ ]:
history_50 = model_50.fit(X, Y_voltage, epochs=50, batch_size=16, validation_split=0.2)
history_50_v2 = model_50_v2.fit(X, Y_voltage, epochs=50, batch_size=16, validation_split=0.2)
history_200 = model_200.fit(X, Y_voltage, epochs=200, batch_size=16, validation_split=0.2)
model_power_flow.fit(X, Y_power_flow, epochs=200, batch_size=16, validation_split=0.2)

## Step 5. Predict results for new input data

In [ ]:
pp.runpp(net)  # Ensure power flow is updated
X_test = np.column_stack([
    net.res_bus.vm_pu.values,
    net.load.p_mw.reindex(net.bus.index, fill_value=0).values,
    net.load.q_mvar.reindex(net.bus.index, fill_value=0).values if 'q_mvar' in net.load else np.zeros(len(net.bus)),
    net.gen.p_mw.reindex(net.bus.index, fill_value=0).values,
    net.gen.q_mvar.reindex(net.bus.index, fill_value=0).values if 'q_mvar' in net.gen else np.zeros(len(net.bus))
]).reshape((X.shape[0], X.shape[1], 1))

predictions_50 = model_50.predict(X_test)
predictions_50_v2 = model_50_v2.predict(X_test)
predictions_200 = model_200.predict(X_test)
predictions_power_flow = model_power_flow.predict(X_test)


## Step 6. Compare predictions for different training epochs and filter sizes.

In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(net.res_bus.vm_pu.values, 'bo--', label='Actual Voltage Magnitudes')
plt.plot(predictions_50.flatten(), 'r*-', label='Predicted (Epochs=50, 24 filters)')
plt.plot(predictions_50_v2.flatten(), 'g^-', label='Predicted (Epochs=50, 240 filters)')
plt.plot(predictions_200.flatten(), 'y+-', label='Predicted (Epochs=200, 240 filters)')
plt.xlabel('Bus Index')
plt.ylabel('Voltage Magnitude (pu)')
plt.title('Comparison of Actual and Predicted Voltage Magnitudes for Different Epochs and Filters')
# Set x-axis labels as 1, 2, 3, 4, 5 for bus indices
plt.xticks(ticks=[0, 1, 2, 3, 4], labels=[1, 2, 3, 4, 5])

plt.legend()
plt.show()


print("Predictions for Epochs=50 (24 filters):", predictions_50)
print("Predictions for Epochs=50 (240 filters):", predictions_50_v2)
print("Predictions for Epochs=200 (240 filters):", predictions_200)

In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(net.res_line.p_from_mw.values[:5], 'bo--', label='Actual P (MW)')
plt.plot(predictions_power_flow[:, 0], 'r*-', label='Predicted P (MW)')
plt.xlabel('Line Index')
plt.ylabel('Active Power Flow (MW)')
plt.legend()
plt.title('Active Power Flow Prediction')
plt.show()

plt.figure(figsize=(10, 5))
plt.plot(net.res_line.q_from_mvar.values[:5], 'bo--', label='Actual Q (MVAR)')
plt.plot(predictions_power_flow[:, 1], 'r*-', label='Predicted Q (MVAR)')
plt.xlabel('Line Index')
plt.ylabel('Reactive Power Flow (MVAR)')
plt.legend()
plt.title('Reactive Power Flow Prediction')
plt.show()

# Print Predictions
print("Predicted Active and Reactive Power Flows:", predictions_power_flow)